In [2]:
import cv2
import numpy as np
import time

def test_frame_stutter(device_number=0, max_frames=500):
    """
    フレーム差分・タイムスタンプ・FPS計測でコマ飛びチェック
    """
    cap = cv2.VideoCapture(device_number)
    if not cap.isOpened():
        print("カメラを開けません")
        return

    # 初期フレーム取得
    ret, prev_frame = cap.read()
    if not ret:
        print("最初のフレーム取得失敗")
        return

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    prev_time = time.time()

    frame_count = 0
    fps_count = 0
    fps_start = time.time()

    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            continue

        # --- フレーム間差分 ---
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        diff = cv2.absdiff(gray, prev_gray)
        diff_flag = "〇" if diff.max() > 5 else "×"

        # --- フレーム間時間 ---
        now = time.time()
        interval = now - prev_time
        prev_time = now

        # --- 実測FPS計測 ---
        fps_count += 1
        if now - fps_start >= 1.0:
            print(f"[FPS] {fps_count} fps")
            fps_count = 0
            fps_start = now

        # --- 表示 ---
        display = frame.copy()
        cv2.putText(display, f"Diff: {diff_flag}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(display, f"Interval: {interval*1000:.1f} ms", (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Stutter Test", display)

        prev_gray = gray
        frame_count += 1

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    print("テスト終了")

# 使用例
test_frame_stutter(device_number=0, max_frames=500)


[FPS] 31 fps
[FPS] 31 fps
[FPS] 30 fps
[FPS] 30 fps
[FPS] 30 fps
[FPS] 31 fps
[FPS] 31 fps
[FPS] 31 fps
[FPS] 30 fps
[FPS] 31 fps
[FPS] 30 fps
[FPS] 31 fps
[FPS] 26 fps
[FPS] 31 fps
[FPS] 30 fps
[FPS] 31 fps
テスト終了
